# Baselines Predictions

Single Step: https://www.theclickreader.com/single-step-time-series-forecasting/

Baseline Method: https://towardsdatascience.com/baseline-models-your-guide-for-model-building-1ec3aa244b8d

- Persistance Algorith: https://machinelearningmastery.com/persistence-time-series-forecasting-with-python/

# <span style='background :khaki' > Install & Importa Libraries </span>

In [1]:
### ***Enviroment Preparation***
# Install Pandas
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install lightgbm

!pip install sktime
!pip install sklego

#!pip install skforecast

# Update pip -- WARNING Resolution
!python.exe -m pip install --upgrade pip

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from datetime import datetime
from datetime import timedelta

from sklego.preprocessing import RepeatingBasisFunction

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

import time

# Paths
data = r"C:\Users\ghoyo\Desktop\TFM 3.0\Project\GeneratedDfs\dayly_stational.json"

import warnings
warnings.filterwarnings("ignore")

#### <span style="background:skyblue"> Load Data <span>

In [3]:
df = pd.read_json(data)

In [4]:
TARGET = ['day_n_streams', 'day_n_songs', 'day_n_podcast', 'day_new_stream', 'day_new_song', 'day_new_podcast']

#### <span style="background:skyblue"> Little Analysis <span>

Tenemos 6 variables a predecir:
- Nº Streams
- Nº Songs
- Nº Podcasts


- Nº New Streams
- Nº New Songs
- Nº New Podcasts


La correlacion entre estas variables es muy alta, por lo que usar alguna para predecir otra sería un 'fake feature'
Por ello, necesitamos predecirlas por separado.


En cuanto a Nº Streams y Nº (tanto como para news), Se utilizará de baseline siempre predecir la media. Y en cuanto a Nº Podcasts se utilizará de Baseline que se escucharán 0.

In [5]:
df[TARGET].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2061 entries, 2017-04-04 to 2022-11-24
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   day_n_streams    2061 non-null   int64
 1   day_n_songs      2061 non-null   int64
 2   day_n_podcast    2061 non-null   int64
 3   day_new_stream   2061 non-null   int64
 4   day_new_song     2061 non-null   int64
 5   day_new_podcast  2061 non-null   int64
dtypes: int64(6)
memory usage: 112.7 KB


In [6]:
df[TARGET].describe()

,day_n_streams,day_n_songs,day_n_podcast,day_new_stream,day_new_song,day_new_podcast
count,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000
mean,39.800097,39.741388,0.058709,6.515284,6.498787,0.016497
std,28.399624,28.401398,0.666791,9.862028,9.855146,0.145213
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,20.000000,0.000000,0.000000,0.000000,0.000000
50%,35.000000,35.000000,0.000000,3.000000,3.000000,0.000000
75%,53.000000,53.000000,0.000000,9.000000,9.000000,0.000000
max,247.000000,247.000000,22.000000,106.000000,106.000000,2.000000


<span style='background :yellow' > Aún sin realizar el proceso se asume que el método de los podcast tiene mejor baseline que predicción, por lo que no se realizara una prediccion directa de sus variables </span>
    
    
<span style='background :yellow' > En cuanto al número de streams y canciones (nuevas o totales), se asume que es mejor la prediccion obtenida con machine learning </span>


Como la correlación de estas dos variables es muy muy alta, de hecho en este dataset concreto es de uno. para otro usuario puede pasar lo mismo con podcast. La candición para tomar la siguiente decisión vendra determinada por los errores de las predicciones, mientras tante, se asume, que el resultado de las predicciones para obtener el número de podcasts va a ser:


Pred Nº Podcasts = Nº Streams - Nº songs


Pred Nº New Podcasts = Nº New Streams - Nº New songs

Streams to classifie (In Future): Nº Streams - Nº New Streams --> We cant predict what we dont know

### <span style="background:lightgreen"> Split Dfs <span>

In [7]:
df_name_list = []

for i in TARGET:
    exec("df_" + i + " = df[i]")
    df_name_list.append('df_' + i)

In [8]:
dataframes = {}

for i in df_name_list:
    dataframes[i] = eval(i).to_frame()

In [9]:
list(dataframes.keys())

['df_day_n_streams',
 'df_day_n_songs',
 'df_day_n_podcast',
 'df_day_new_stream',
 'df_day_new_song',
 'df_day_new_podcast']

# <span style='background :khaki' > Baseline Method </span>

### <span style="background:lightgreen"> Dummy Regresors - lag_1 <span>

In [10]:
for d in dataframes:
    dataframes[d]['t-1'] = dataframes[d][d.replace('df_', '')].shift(1)
    dataframes[d].dropna(inplace = True)
    dataframes[d]['i'] = [i for i in range(dataframes[d].shape[0])]
    print(dataframes[d].head())

            day_n_streams    t-1  i
2017-04-05            123   66.0  0
2017-04-06             63  123.0  1
2017-04-07             45   63.0  2
2017-04-08             56   45.0  3
2017-04-09             36   56.0  4
            day_n_songs    t-1  i
2017-04-05          123   66.0  0
2017-04-06           63  123.0  1
2017-04-07           45   63.0  2
2017-04-08           56   45.0  3
2017-04-09           36   56.0  4
            day_n_podcast  t-1  i
2017-04-05              0  0.0  0
2017-04-06              0  0.0  1
2017-04-07              0  0.0  2
2017-04-08              0  0.0  3
2017-04-09              0  0.0  4
            day_new_stream  t-1  i
2017-04-05               0  0.0  0
2017-04-06               0  0.0  1
2017-04-07               0  0.0  2
2017-04-08               0  0.0  3
2017-04-09               0  0.0  4
            day_new_song  t-1  i
2017-04-05             0  0.0  0
2017-04-06             0  0.0  1
2017-04-07             0  0.0  2
2017-04-08             0  0.0  3
2

#### <span style="background:skyblue"> Split Data in Train and Test <span>

In [11]:
X_l = []
y_l = []

X_d = {}
y_d = {}

for d,i in zip(dataframes, range(len(dataframes))):
    exec("X_" + str(d.replace('df_','')) + " = dataframes[d]['t-1']")
    exec("y_" + str(d.replace('df_','')) + " = dataframes[d][d.replace('df_', '')]")
    X_l.append('X_' + str(d.replace('df_','')))
    y_l.append('y_' + str(d.replace('df_','')))
    
for x,y in zip(X_l, y_l):
    X_d[x] = eval(x)
    y_d[y] = eval(y)

<span style="background:violet"> Persistance Methods Function <span>

In [12]:
# persistence model
def model_persistence(x):
    return x

#### <span style="background:skyblue"> Predict <span>

In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# Evaluate Forecast
test_size = 1
predictions = []
ma = []
ms = []

for xd,yd,d in zip(X_d, y_d, dataframes):
    predict = []
    for x in X_d[xd][-test_size:]:
        yhat = model_persistence(x)
        predict.append(yhat)
        
    mse = mean_squared_error(y_d[yd][-test_size:], predict)
    mae = mean_absolute_error(y_d[yd][-test_size:], predict)
    #r2 = r2_score(y_d[yd][-test_size:], predict)
    print(d)
    
    print('\nPrediction Vs Original')
    print('\t', str(predict).replace('[', '').replace(']',''), '--', int(y_d[yd][-test_size:].values), '\n')
    
    print('Test MSE: %.3f' % mse)
    print('Test MAE: %.3f' % mae)
    #print('Test MAE: %.3f' % r2)
    predictions.append(predict)
    ma.append(mae)
    ms.append(mse)
    
    print('---------------------------------------\n')

df_day_n_streams

Prediction Vs Original
	 21.0 -- 19 

Test MSE: 4.000
Test MAE: 2.000
---------------------------------------

df_day_n_songs

Prediction Vs Original
	 21.0 -- 19 

Test MSE: 4.000
Test MAE: 2.000
---------------------------------------

df_day_n_podcast

Prediction Vs Original
	 0.0 -- 0 

Test MSE: 0.000
Test MAE: 0.000
---------------------------------------

df_day_new_stream

Prediction Vs Original
	 1.0 -- 5 

Test MSE: 16.000
Test MAE: 4.000
---------------------------------------

df_day_new_song

Prediction Vs Original
	 1.0 -- 5 

Test MSE: 16.000
Test MAE: 4.000
---------------------------------------

df_day_new_podcast

Prediction Vs Original
	 0.0 -- 0 

Test MSE: 0.000
Test MAE: 0.000
---------------------------------------



In [14]:
print(df.columns)
print('mae: ', ma)
print('mse: ', ms)

Index(['day_n_streams', 'day_n_songs', 'day_n_podcast', 'day_new_stream',
       'day_new_song', 'day_new_podcast'],
      dtype='object')
mae:  [2.0, 2.0, 0.0, 4.0, 4.0, 0.0]
mse:  [4.0, 4.0, 0.0, 16.0, 16.0, 0.0]


<span style="background:violet"> Evaluation Methods <span>

https://sitiobigdata.com/2018/08/27/machine-learning-metricas-regresion-mse/#

#### This Only Evaluates a single value, we need to apply a validation data set 

### <span style="background:lightgreen"> Cross Validation <span>

https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4

https://goldinlocks.github.io/Time-Series-Cross-Validation/

https://www.sciencedirect.com/science/article/abs/pii/S0020025511006773

#### <span style="background:skyblue"> TSCV - Time Serie Cross Validation <span>

Train size = 80%
Validation size = 10%
Test size = 10% -- In this case we are only planning to predict one step ahed so this changes

In [15]:
df.shape[0]*15/100

309.15

In [21]:
%%time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

c = 0
test_size = 1

predictions = []
cv_b_mse = []
cv_b_mae = []

for xd,yd,d in zip(X_d, y_d, dataframes):
    cc = 0
    predict = []
    b_mse = []
    b_mae = []
    
    tscv = TimeSeriesSplit(max_train_size=None, n_splits = 22, test_size=test_size)
    #print(tscv)

    print('__________________________________', d, '_____________________________________\n')
    
    for train_index, test_index in tscv.split(X_d[xd]):
        pre = []
        
        print('TRAIN:', train_index, 'TEST:', test_index) 
        print('type', type(train_index), type(test_index))
        #print(type(X_d[xd][train_index]))
        #print(X_d[xd][train_index])
        X_train, X_test = X_d[xd][train_index], X_d[xd][test_index]
        y_train, y_test = y_d[yd][train_index], y_d[yd][test_index]

        for x in X_test:
            yhat = model_persistence(x)
            pre.append(yhat)
            print('\nX_test = ', x, '-- Predict = ', yhat, '\n')
        
        for x in test_index:
            print('T = ', dataframes[d].loc[dataframes[d]['i'] == x], '\nT-1', dataframes[d].loc[dataframes[d]['i'] == x-1])
            
        predict.append(pre)
        
        b_mse.append(mean_squared_error(y_test, pre))
        b_mae.append(mean_absolute_error(y_test, pre))
        print('\nBaseline_MSE = ', b_mse[cc], '-- Baseline_MAE = ', b_mae[cc])
        
        cc = cc+1
        
        print('\n------------------------------------------------------------------------\n')
    
    predictions.append(predict)
    
    cv_b_mse.append(np.mean(b_mse))
    cv_b_mae.append(np.mean(b_mae))
    print('\nMSE = ', cv_b_mse[c])
    print('\nMAE = ', cv_b_mae[c])
    
    c = c+1
    
    print('_______________________________________________________________________________________')
    print('_______________________________________________________________________________________\n')

__________________________________ df_day_n_streams _____________________________________

TRAIN: [   0    1    2 ... 2035 2036 2037] TEST: [2038]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  16.0 -- Predict =  16.0 

T =              day_n_streams   t-1     i
2022-11-03             15  16.0  2038 
T-1             day_n_streams   t-1     i
2022-11-02             16  15.0  2037

Baseline_MSE =  1.0 -- Baseline_MAE =  1.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2036 2037 2038] TEST: [2039]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  15.0 -- Predict =  15.0 

T =              day_n_streams   t-1     i
2022-11-04             62  15.0  2039 
T-1             day_n_streams   t-1     i
2022-11-03             15  16.0  2038

Baseline_MSE =  2209.0 -- Baseline_MAE =  47.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2037 2038 2039] TEST: [2

T =              day_n_songs  t-1     i
2022-11-12           65  1.0  2047 
T-1             day_n_songs   t-1     i
2022-11-11            1  13.0  2046

Baseline_MSE =  4096.0 -- Baseline_MAE =  64.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2045 2046 2047] TEST: [2048]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  65.0 -- Predict =  65.0 

T =              day_n_songs   t-1     i
2022-11-13           12  65.0  2048 
T-1             day_n_songs  t-1     i
2022-11-12           65  1.0  2047

Baseline_MSE =  2809.0 -- Baseline_MAE =  53.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2046 2047 2048] TEST: [2049]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  12.0 -- Predict =  12.0 

T =              day_n_songs   t-1     i
2022-11-14            0  12.0  2049 
T-1             day_n_songs   t-1     i
2022-11-13           12  65.0  2048

Bas

2022-11-19              0  0.0  2054

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2053 2054 2055] TEST: [2056]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_n_podcast  t-1     i
2022-11-21              0  0.0  2056 
T-1             day_n_podcast  t-1     i
2022-11-20              0  0.0  2055

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2054 2055 2056] TEST: [2057]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_n_podcast  t-1     i
2022-11-22              0  0.0  2057 
T-1             day_n_podcast  t-1     i
2022-11-21              0  0.0  2056

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

TR


Baseline_MSE =  4.0 -- Baseline_MAE =  2.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2041 2042 2043] TEST: [2044]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_new_song  t-1     i
2022-11-09             4  0.0  2044 
T-1             day_new_song  t-1     i
2022-11-08             0  2.0  2043

Baseline_MSE =  16.0 -- Baseline_MAE =  4.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2042 2043 2044] TEST: [2045]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  4.0 -- Predict =  4.0 

T =              day_new_song  t-1     i
2022-11-10             0  4.0  2045 
T-1             day_new_song  t-1     i
2022-11-09             4  0.0  2044

Baseline_MSE =  16.0 -- Baseline_MAE =  4.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2043 2044 2045] TE


Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2053 2054 2055] TEST: [2056]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_new_podcast  t-1     i
2022-11-21                0  0.0  2056 
T-1             day_new_podcast  t-1     i
2022-11-20                0  0.0  2055

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 ... 2054 2055 2056] TEST: [2057]
type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_new_podcast  t-1     i
2022-11-22                0  0.0  2057 
T-1             day_new_podcast  t-1     i
2022-11-21                0  0.0  2056

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

TRAIN: [   0    1    2 

In [22]:
for i,mae,mse in zip(dataframes, cv_b_mae, cv_b_mse):
    print(i)
    print('\tMAE: ', mae)
    print('\tMSE: ', np.round(mse, 3))

df_day_n_streams
	MAE:  22.681818181818183
	MSE:  1032.864
df_day_n_songs
	MAE:  22.59090909090909
	MSE:  1037.955
df_day_n_podcast
	MAE:  0.45454545454545453
	MSE:  1.273
df_day_new_stream
	MAE:  7.045454545454546
	MSE:  129.318
df_day_new_song
	MAE:  7.136363636363637
	MSE:  131.045
df_day_new_podcast
	MAE:  0.09090909090909091
	MSE:  0.091


#### <span style="background:skyblue"> B-CV - Block Cross Validation <span>

<span style="background:violet"> BCV Function <span>

In [23]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.999999 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4

In [24]:
%%time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

c = 0
test_size = 1

predictions = []
cv_b_mse = []
cv_b_mae = []

for xd,yd,d in zip(X_d, y_d, dataframes):
    cc = 0
    predict = []
    b_mse = []
    b_mae = []
    
    tscv = BlockingTimeSeriesSplit(n_splits = 22)
    #print(tscv)

    print('__________________________________', d, '_____________________________________\n')
    
    for train_index, test_index in tscv.split(X_d[xd]):
        pre = []
        
        #print('TRAIN:', train_index, 'TEST:', test_index)
        print('type', type(train_index), type(test_index))
        #print(type(X_d[xd][train_index]))
        #print(X_d[xd][train_index])
        X_train, X_test = X_d[xd][train_index], X_d[xd][test_index]
        y_train, y_test = y_d[yd][train_index], y_d[yd][test_index]

        for x in X_test:
            yhat = model_persistence(x)
            pre.append(yhat)
            print('\nX_test = ', x, '-- Predict = ', yhat, '\n')
        
        for x in test_index:
            print('T = ', dataframes[d].loc[dataframes[d]['i'] == x], '\nT-1', dataframes[d].loc[dataframes[d]['i'] == x-1])
            
        predict.append(pre)
        
        b_mse.append(mean_squared_error(y_test, pre))
        b_mae.append(mean_absolute_error(y_test, pre))
        print('\nBaseline_MSE = ', b_mse[cc], '-- Baseline_MAE = ', b_mae[cc])
        
        cc = cc+1
        
        print('\n------------------------------------------------------------------------\n')
    
    predictions.append(predict)
    
    cv_b_mse.append(np.mean(b_mse))
    cv_b_mae.append(np.mean(b_mae))
    print('\nMSE = ', cv_b_mse[c])
    print('\nMAE = ', cv_b_mae[c])
    
    c = c+1
    
    print('_______________________________________________________________________________________')
    print('_______________________________________________________________________________________\n')



__________________________________ df_day_n_streams _____________________________________

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  45.0 -- Predict =  45.0 

T =              day_n_streams   t-1   i
2017-07-06              6  45.0  92 
T-1             day_n_streams   t-1   i
2017-07-05             45  87.0  91

Baseline_MSE =  1521.0 -- Baseline_MAE =  39.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  97.0 -- Predict =  97.0 

T =              day_n_streams   t-1    i
2017-10-07             25  97.0  185 
T-1             day_n_streams   t-1    i
2017-10-06             97  28.0  184

Baseline_MSE =  5184.0 -- Baseline_MAE =  72.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  16.0 -- Predict =  16.0 

T =              day_n_streams   t-1    i
2018-01-08             40  16.0  278 
T

2018-10-14           22  39.0  557 
T-1             day_n_songs   t-1    i
2018-10-13           39  24.0  556

Baseline_MSE =  289.0 -- Baseline_MAE =  17.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  30.0 -- Predict =  30.0 

T =              day_n_songs   t-1    i
2019-01-15           96  30.0  650 
T-1             day_n_songs   t-1    i
2019-01-14           30  18.0  649

Baseline_MSE =  4356.0 -- Baseline_MAE =  66.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  40.0 -- Predict =  40.0 

T =              day_n_songs   t-1    i
2019-04-18           65  40.0  743 
T-1             day_n_songs   t-1    i
2019-04-17           40  70.0  742

Baseline_MSE =  625.0 -- Baseline_MAE =  25.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.

T =              day_n_podcast  t-1     i
2020-07-26              0  0.0  1208 
T-1             day_n_podcast  t-1     i
2020-07-25              0  0.0  1207

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_n_podcast  t-1     i
2020-10-27              0  0.0  1301 
T-1             day_n_podcast  t-1     i
2020-10-26              0  0.0  1300

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_n_podcast  t-1     i
2021-01-28              0  0.0  1394 
T-1             day_n_podcast  t-1     i
2021-01-27              0  0.0  1393

Baseline_MSE =  0.0 -- Baseline_MAE =  0.0

-------------------------------------------------------------------

T =              day_new_song  t-1    i
2019-07-20             2  6.0  836 
T-1             day_new_song  t-1    i
2019-07-19             6  0.0  835

Baseline_MSE =  16.0 -- Baseline_MAE =  4.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  2.0 -- Predict =  2.0 

T =              day_new_song  t-1    i
2019-10-21             3  2.0  929 
T-1             day_new_song  t-1    i
2019-10-20             2  0.0  928

Baseline_MSE =  1.0 -- Baseline_MAE =  1.0

------------------------------------------------------------------------

type <class 'numpy.ndarray'> <class 'numpy.ndarray'>

X_test =  0.0 -- Predict =  0.0 

T =              day_new_song  t-1     i
2020-01-22             5  0.0  1022 
T-1             day_new_song  t-1     i
2020-01-21             0  1.0  1021

Baseline_MSE =  25.0 -- Baseline_MAE =  5.0

------------------------------------------------------------------------

type <class

In [25]:
for i,mae,mse in zip(dataframes, cv_b_mae, cv_b_mse):
    print(i)
    print('\tMAE: ', mae)
    print('\tMSE: ', np.round(mse, 3))


df_day_n_streams
	MAE:  20.454545454545453
	MSE:  787.545
df_day_n_songs
	MAE:  20.454545454545453
	MSE:  787.182
df_day_n_podcast
	MAE:  0.09090909090909091
	MSE:  0.091
df_day_new_stream
	MAE:  5.954545454545454
	MSE:  56.773
df_day_new_song
	MAE:  5.954545454545454
	MSE:  56.773
df_day_new_podcast
	MAE:  0.0
	MSE:  0.0


Como se observa, es mejor predecir 0, o el resultado del día anterior en cuanto a los datasets de podcasts. Este caso puede ser diferente para otros usuarios, de hecho puede llegar a ser contrario, de un modo que esto mismo ocurra para las canciones.

Esto significa que este tipo de usuario utiliza su cuenta para una única actividad en vez de para las dos.

Como el método base obtiene unos resutlados muy complicados de mejorar, con muy poca computación. Se define como el método adecuado para predecir estas variables.